# Дообучение XTTS-v2

In [1]:
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 30.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 91.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 8.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 406.3 kB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━

In [4]:
!pip install transformers==4.37.1

In [3]:
!pip install git+https://github.com/coqui-ai/TTS

  Cloning https://github.com/coqui-ai/TTS to /tmp/pip-req-build-b46ru7j5
  Running command git clone --filter=blob:none --quiet https://github.com/coqui-ai/TTS /tmp/pip-req-build-b46ru7j5
  Resolved https://github.com/coqui-ai/TTS to commit dbf1a08a0d4e47fdad6172e433eeb34bc6b13b4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 614.5 kB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 8.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 42.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip show torch

Name: torch
Version: 2.5.1+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, easyocr, encodec, fastai, kornia, peft, pytorch-ignite, pytorch-lightning, sentence-transformers, stable-baselines3, timm, torchaudio, torchdata, torchmetrics, torchvision, trainer, TTS


In [21]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

In [22]:
OUT_PATH = '/kaggle/working/run/'
os.makedirs(OUT_PATH, exist_ok=True)

CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)

In [8]:
# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)

# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )

 > Downloading DVAE files!


  0%|          | 0.00/1.07k [00:00<?, ?iB/s]
100%|██████████| 1.07k/1.07k [00:00<00:00, 1.28kiB/s]

  5%|▍         | 9.73M/211M [00:00<00:02, 97.3MiB/s]
 10%|█         | 21.1M/211M [00:00<00:01, 107MiB/s] 
 15%|█▌        | 32.6M/211M [00:00<00:01, 110MiB/s]
 21%|██        | 43.6M/211M [00:00<00:01, 106MiB/s]
 26%|██▋       | 55.5M/211M [00:00<00:01, 110MiB/s]
 32%|███▏      | 67.1M/211M [00:00<00:01, 112MiB/s]
 38%|███▊      | 79.0M/211M [00:00<00:01, 114MiB/s]
 43%|████▎     | 90.9M/211M [00:00<00:01, 116MiB/s]
 49%|████▉     | 103M/211M [00:00<00:00, 117MiB/s] 
 54%|█████▍    | 115M/211M [00:01<00:00, 117MiB/s]
 60%|██████    | 126M/211M [00:01<00:00, 115MiB/s]
 66%|██████▌   | 138M/211M [00:01<00:00, 110MiB/s]
 71%|███████   | 149M/211M [00:01<00:00, 105MiB/s]
 76%|███████▌  | 160M/211M [00:01<00:00, 107MiB/s]
 81%|████████  | 171M/211M [00:01<00:00, 106MiB/s]
 86%|████████▌ | 181M/211M [00:01<00:00, 105MiB/s]
 92%|█████████▏| 193M/211M [00:01<00:00, 109MiB/s]
 97%|█████████▋| 205M/

 > Downloading XTTS v2.0 files!


100%|██████████| 211M/211M [00:02<00:00, 88.0MiB/s]

361kiB [00:00, 415kiB/s]87G [00:00<?, ?iB/s]

  1%|          | 9.57M/1.87G [00:00<00:19, 95.7MiB/s]
  1%|          | 21.0M/1.87G [00:00<00:17, 107MiB/s] 
  2%|▏         | 31.7M/1.87G [00:00<00:18, 100MiB/s]
  2%|▏         | 41.7M/1.87G [00:00<00:18, 96.9MiB/s]
  3%|▎         | 53.4M/1.87G [00:00<00:17, 104MiB/s] 
  3%|▎         | 65.1M/1.87G [00:00<00:16, 108MiB/s]
  4%|▍         | 77.0M/1.87G [00:00<00:16, 112MiB/s]
  5%|▍         | 88.2M/1.87G [00:00<00:16, 107MiB/s]
  5%|▌         | 100M/1.87G [00:00<00:16, 110MiB/s] 
  6%|▌         | 111M/1.87G [00:01<00:16, 106MiB/s]
  7%|▋         | 123M/1.87G [00:01<00:15, 109MiB/s]
  7%|▋         | 135M/1.87G [00:01<00:15, 112MiB/s]
  8%|▊         | 146M/1.87G [00:01<00:15, 113MiB/s]
  8%|▊         | 158M/1.87G [00:01<00:14, 114MiB/s]
  9%|▉         | 170M/1.87G [00:01<00:14, 115MiB/s]
 10%|▉         | 182M/1.87G [00:01<00:14, 116MiB/s]
 10%|█         | 193M/1.87G [00:01<00:14, 117MiB/s]
 11%

In [9]:
training_dir = "/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1"


100%|██████████| 1.87G/1.87G [00:35<00:00, 102MiB/s]

In [23]:
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  # for multi-gpu training please make it False
START_WITH_EVAL = True  # if True it will star with evaluation
BATCH_SIZE = 2  # set here the batch size
GRAD_ACUMM_STEPS = 64  # set here the grad accumulation steps
LANGUAGE = "en"

## Конфигурация датасета

In [24]:
config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="ljspeech",
    path="/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1",
    meta_file_train="/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/metadata.csv",
    language="en",
)

DATASETS_CONFIG_LIST = [config_dataset]

In [25]:
# Training sentences generations
SPEAKER_REFERENCE = [
    "/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs/LJ001-0001.wav"  # speaker reference to be used in training test sentences
]
LANGUAGE = config_dataset.language

## Аргументы модели

In [26]:
model_args = GPTArgs(
        max_conditioning_length=198450,  # 9 secs
        min_conditioning_length=66150,  # 3 secs
        debug_loading_failures=False,
        max_wav_length=255995,  # ~11.6 seconds
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )

In [27]:
audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)

## Конфигурация обучения

In [28]:
# Logging parameters
RUN_NAME = "GPT_XTTS_v2.0_LJSpeech_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

In [29]:
config = GPTTrainerConfig(
        epochs = 5,
        run_eval = True,
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=4,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=5000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        # target_loss="loss",
        print_eval=False,
        # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # learning rate
        lr_scheduler="MultiStepLR",
        # it was adjusted accordly for the new step scheme
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
            {
                "text": "It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "This cake is great. It's so delicious and moist.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
        ],
    )

model = GPTTrainer.init_from_config(config)


>> DVAE weights restored from: /kaggle/working/run/XTTS_v2.0_original_model_files/dvae.pth


In [30]:
train_samples, eval_samples = load_tts_samples(config_dataset, eval_split=True, eval_split_size=0.02)

 | > Found 13100 files in /kaggle/input/the-lj-speech-dataset/LJSpeech-1.1


# Обучение

In [31]:
trainer = Trainer(
    TrainerArgs(
        restore_path=None,
        skip_train_epoch=False,
        start_with_eval=START_WITH_EVAL,
        grad_accum_steps=GRAD_ACUMM_STEPS,
    ),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
trainer.fit()

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 4
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
2025-07-17 10:49:37.872889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752749378.047516      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

 > Filtering invalid eval samples!!



 > EVALUATION 



 > Total eval samples after filtering: 262
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.058132039583646336 (+0)
     | > avg_loss_text_ce: 0.02300384864211083 (+0)
     | > avg_loss_mel_ce: 3.6739199308248667 (+0)
     | > avg_loss: 3.696923776773306 (+0)


 > EPOCH: 1/5
 --> /kaggle/working/run/GPT_XTTS_v2.0_LJSpeech_FT-July-17-2025_10+49AM-0000000

 > TRAINING (2025-07-17 10:50:31) 


 > Sampling by language: dict_keys(['en'])



   --> TIME: 2025-07-17 10:50:33 -- STEP: 0/6419 -- GLOBAL_STEP: 0
     | > loss_text_ce: 0.02335914969444275  (0.02335914969444275)
     | > loss_mel_ce: 3.715486764907837  (3.715486764907837)
     | > loss: 0.05841946601867676  (0.05841946601867676)
     | > current_lr: 5e-06 
     | > step_time: 0.3312  (0.3312394618988037)
     | > loader_time: 1.4349  (1.4348711967468262)


   --> TIME: 2025-07-17 10:50:52 -- STEP: 50/6419 -- GLOBAL_STEP: 50
     | > loss_text_ce: 0.022206701338291168  (0.024238972142338753)
     | > loss_mel_ce: 3.8040578365325928  (3.7184663009643555)
     | > loss: 0.0597853846848011  (0.05847976982593536)
     | > current_lr: 5e-06 
     | > step_time: 0.2644  (0.24202992439270019)
     | > loader_time: 0.0087  (0.010888595581054688)


   --> TIME: 2025-07-17 10:51:11 -- STEP: 100/6419 -- GLOBAL_STEP: 100
     | > loss_text_ce: 0.020164813846349716  (0.02376967852935195)
     | > loss_mel_ce: 3.466541051864624  (3.7050228404998777)
     | > loss: 0.0544797778

RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 5510800128 vs 5510800024